In [1]:
import os
import subprocess

import pandas as pd

In [2]:
base_dir = '../data/'

In [3]:
df_resolutions = pd.read_csv(base_dir + "resolucion.csv")
df_resolutions.head()

,Expediente,Fecha resolucion,Sala,PDF link,PDF nombre,Resuelve texto,Firma,Cargo,Firma.1,Cargo.1
0,CFP 14216/2003,03/08/2017,CAMARA CRIMINAL Y CORRECCIONAL FEDERAL - SALA 1,http://www.cij.gov.ar/scp/include/showFile.php...,5574569.pdf,RESUELVE: CONCEDER el recurso de casación inte...,JORGE LUIS BALLESTERO,JUEZ DE CÁMARA,LEOPOLDO OSCAR BRUGLIA,JUEZ DE CÁMARA
1,CFP 14216/2003,03/08/2017,CAMARA CRIMINAL Y CORRECCIONAL FEDERAL - SALA 1,http://www.cij.gov.ar/scp/include/showFile.php...,5574569.pdf,RESUELVE: CONCEDER el recurso de casación inte...,JORGE LUIS BALLESTERO,JUEZ DE CÁMARA,JORGE LUIS BALLESTERO,JUEZ DE CÁMARA
2,CFP 14216/2003,03/08/2017,CAMARA CRIMINAL Y CORRECCIONAL FEDERAL - SALA 1,http://www.cij.gov.ar/scp/include/showFile.php...,5574569.pdf,RESUELVE: CONCEDER el recurso de casación inte...,JORGE LUIS BALLESTERO,JUEZ DE CÁMARA,DARIO ANIBAL POZZI,SECRETARIO
3,CFP 14216/2003,03/08/2017,CAMARA CRIMINAL Y CORRECCIONAL FEDERAL - SALA 1,http://www.cij.gov.ar/scp/include/showFile.php...,5574569.pdf,RESUELVE: CONCEDER el recurso de casación inte...,DARIO ANIBAL POZZI,SECRETARIO,LEOPOLDO OSCAR BRUGLIA,JUEZ DE CÁMARA
4,CFP 14216/2003,03/08/2017,CAMARA CRIMINAL Y CORRECCIONAL FEDERAL - SALA 1,http://www.cij.gov.ar/scp/include/showFile.php...,5574569.pdf,RESUELVE: CONCEDER el recurso de casación inte...,DARIO ANIBAL POZZI,SECRETARIO,JORGE LUIS BALLESTERO,JUEZ DE CÁMARA


In [4]:
total = df_resolutions.shape[0]
bash_command = "pdftotext -layout -enc UTF-8 {filename}"

for index, row in df_resolutions.iterrows():
    # progress text
    text = "\rCombirtiendo a txt el PDF {0} de {1}"
    print(text.format(index + 1, total), end='')
    
    command = bash_command.format(filename = base_dir + 'PDFs/' + row['PDF nombre'])
    process = subprocess.Popen(command.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
    
    if error:
        print(row['PDF nombre'], error)

Combirtiendo a txt el PDF 18937 de 18937

In [5]:
def find_paragraph(row):
    # No se por que, pero un pdf esta vacio. Lo ignoro
    if os.stat(base_dir + 'PDFs/' + row['PDF nombre']).st_size == 0:
        #if row['PDF nombre'] == '4835623.pdf':
        return
    
    file_name = row['PDF nombre'].replace('.pdf', '.txt')
    
    # Read the file
    with open(base_dir + 'PDFs/' + file_name) as file:
        lines = [line.replace('\n', '') for line in file.readlines()]

    # Remove the PDF header
    for index, line in enumerate(lines):
        if '////nos Aires,' in line:
            head_end = index + 1
            lines = lines[head_end:]
            break
    
    # Remove the PDF footer
    # hago en max en caso de que existan muchos 'Registrese,'
    footer_start = [index for index, line in enumerate(lines) 
                    if ' '.join(line.split()).startswith('Regístrese,')]
    if footer_start:
        lines = lines[:max(footer_start)]

    # Elimino las lineas que comienzan con las siguientes palabras
    remove_keys = ['Poder Judicial de la Nación', 'CAMARA ','Fecha de firma:', 'Alta en sistema:', 
                   'Firmado por:', 'Firmado(ante mi) por:', '#', 'Y VISTOS Y', row['Expediente'],
                   'Año del Bicentenario de la Declaración de la Independencia Nacional']
    lines_swap = []

    for line in lines:
        line_temp = ' '.join(line.split())
        remove_line = any([line_temp.startswith(text) for text in remove_keys])
        if not remove_line and line:
            lines_swap.append(line)

    lines = lines_swap
    
    # join the lines and remove extra spaces
    full_text = ' '.join(lines)
    full_text = ' '.join(full_text.split())

    # After
    after_resolution = ['RESUELVE', 'RESUELVO']

    after_index = max([full_text.find(key) for key in after_resolution])
    if after_index > 0:
        resolution = full_text[after_index:]

    else:
        # Before
        amount_spaces = len(lines[-1]) -  len(' '.join(lines[-1].split()))
        for index, line in enumerate(lines[::-1]):
            spaces = len(line) -  len(' '.join(line.split()))

            if spaces != amount_spaces:
                break

        resolution = ' '.join(lines[len(lines) - index - 1:])
        resolution = ' '.join(resolution.split())

    return resolution

In [6]:
print('Extrayendo resolucion: ...', end='')

resolution_text = df_resolutions.apply(find_paragraph, axis=1)
df_resolutions['Resuelve texto'] = resolution_text
df_resolutions['Resuelve texto'] = df_resolutions['Resuelve texto'].fillna('No texto')

print('\rExtrayendo resolucion: Listo')

df_resolutions.head()

Extrayendo resolucion: Listo


,Expediente,Fecha resolucion,Sala,PDF link,PDF nombre,Resuelve texto,Firma,Cargo,Firma.1,Cargo.1
0,CFP 14216/2003,03/08/2017,CAMARA CRIMINAL Y CORRECCIONAL FEDERAL - SALA 1,http://www.cij.gov.ar/scp/include/showFile.php...,5574569.pdf,RESUELVE: CONCEDER el recurso de casación inte...,JORGE LUIS BALLESTERO,JUEZ DE CÁMARA,LEOPOLDO OSCAR BRUGLIA,JUEZ DE CÁMARA
1,CFP 14216/2003,03/08/2017,CAMARA CRIMINAL Y CORRECCIONAL FEDERAL - SALA 1,http://www.cij.gov.ar/scp/include/showFile.php...,5574569.pdf,RESUELVE: CONCEDER el recurso de casación inte...,JORGE LUIS BALLESTERO,JUEZ DE CÁMARA,JORGE LUIS BALLESTERO,JUEZ DE CÁMARA
2,CFP 14216/2003,03/08/2017,CAMARA CRIMINAL Y CORRECCIONAL FEDERAL - SALA 1,http://www.cij.gov.ar/scp/include/showFile.php...,5574569.pdf,RESUELVE: CONCEDER el recurso de casación inte...,JORGE LUIS BALLESTERO,JUEZ DE CÁMARA,DARIO ANIBAL POZZI,SECRETARIO
3,CFP 14216/2003,03/08/2017,CAMARA CRIMINAL Y CORRECCIONAL FEDERAL - SALA 1,http://www.cij.gov.ar/scp/include/showFile.php...,5574569.pdf,RESUELVE: CONCEDER el recurso de casación inte...,DARIO ANIBAL POZZI,SECRETARIO,LEOPOLDO OSCAR BRUGLIA,JUEZ DE CÁMARA
4,CFP 14216/2003,03/08/2017,CAMARA CRIMINAL Y CORRECCIONAL FEDERAL - SALA 1,http://www.cij.gov.ar/scp/include/showFile.php...,5574569.pdf,RESUELVE: CONCEDER el recurso de casación inte...,DARIO ANIBAL POZZI,SECRETARIO,JORGE LUIS BALLESTERO,JUEZ DE CÁMARA


In [7]:
df_resolutions[~df_resolutions['Resuelve texto'].str.startswith('RESUEL')]

,Expediente,Fecha resolucion,Sala,PDF link,PDF nombre,Resuelve texto,Firma,Cargo,Firma.1,Cargo.1
63,CFP 14216/2003,15/06/2017,CAMARA CRIMINAL Y CORRECCIONAL FEDERAL - SALA 1,http://www.cij.gov.ar/scp/include/showFile.php...,5429254.pdf,"No obstante tal error, el análisis de admisibi...",JORGE LUIS BALLESTERO,JUEZ DE CÁMARA,LEOPOLDO OSCAR BRUGLIA,JUEZ DE CÁMARA
64,CFP 14216/2003,15/06/2017,CAMARA CRIMINAL Y CORRECCIONAL FEDERAL - SALA 1,http://www.cij.gov.ar/scp/include/showFile.php...,5429254.pdf,"No obstante tal error, el análisis de admisibi...",JORGE LUIS BALLESTERO,JUEZ DE CÁMARA,JORGE LUIS BALLESTERO,JUEZ DE CÁMARA
65,CFP 14216/2003,15/06/2017,CAMARA CRIMINAL Y CORRECCIONAL FEDERAL - SALA 1,http://www.cij.gov.ar/scp/include/showFile.php...,5429254.pdf,"No obstante tal error, el análisis de admisibi...",JORGE LUIS BALLESTERO,JUEZ DE CÁMARA,DARIO ANIBAL POZZI,SECRETARIO
66,CFP 14216/2003,15/06/2017,CAMARA CRIMINAL Y CORRECCIONAL FEDERAL - SALA 1,http://www.cij.gov.ar/scp/include/showFile.php...,5429254.pdf,"No obstante tal error, el análisis de admisibi...",DARIO ANIBAL POZZI,SECRETARIO,LEOPOLDO OSCAR BRUGLIA,JUEZ DE CÁMARA
67,CFP 14216/2003,15/06/2017,CAMARA CRIMINAL Y CORRECCIONAL FEDERAL - SALA 1,http://www.cij.gov.ar/scp/include/showFile.php...,5429254.pdf,"No obstante tal error, el análisis de admisibi...",DARIO ANIBAL POZZI,SECRETARIO,JORGE LUIS BALLESTERO,JUEZ DE CÁMARA
68,CFP 14216/2003,15/06/2017,CAMARA CRIMINAL Y CORRECCIONAL FEDERAL - SALA 1,http://www.cij.gov.ar/scp/include/showFile.php...,5429254.pdf,"No obstante tal error, el análisis de admisibi...",DARIO ANIBAL POZZI,SECRETARIO,DARIO ANIBAL POZZI,SECRETARIO
69,CFP 14216/2003,15/06/2017,CAMARA CRIMINAL Y CORRECCIONAL FEDERAL - SALA 1,http://www.cij.gov.ar/scp/include/showFile.php...,5429254.pdf,"No obstante tal error, el análisis de admisibi...",LEOPOLDO OSCAR BRUGLIA,JUEZ DE CÁMARA,LEOPOLDO OSCAR BRUGLIA,JUEZ DE CÁMARA
70,CFP 14216/2003,15/06/2017,CAMARA CRIMINAL Y CORRECCIONAL FEDERAL - SALA 1,http://www.cij.gov.ar/scp/include/showFile.php...,5429254.pdf,"No obstante tal error, el análisis de admisibi...",LEOPOLDO OSCAR BRUGLIA,JUEZ DE CÁMARA,JORGE LUIS BALLESTERO,JUEZ DE CÁMARA
71,CFP 14216/2003,15/06/2017,CAMARA CRIMINAL Y CORRECCIONAL FEDERAL - SALA 1,http://www.cij.gov.ar/scp/include/showFile.php...,5429254.pdf,"No obstante tal error, el análisis de admisibi...",LEOPOLDO OSCAR BRUGLIA,JUEZ DE CÁMARA,DARIO ANIBAL POZZI,SECRETARIO
596,CFP 14216/2003,13/05/2016,CAMARA CRIMINAL Y CORRECCIONAL FEDERAL - SALA 1,http://www.cij.gov.ar/scp/include/showFile.php...,4236880.pdf,"Atento que en el día de la fecha, en el marco ...",JORGE LUIS BALLESTERO,JUEZ DE CÁMARA,DARIO ANIBAL POZZI,SECRETARIO


In [8]:
df_resolutions.to_csv(base_dir + 'resolucion.csv', index=False)

# PDF Invalidos

In [13]:
import shutil

In [12]:
df_filter = df_resolutions[~df_resolutions['Resuelve texto'].str.startswith('RESUEL')]
pdfs = df_filter['PDF nombre'].unique()
len(pdfs)

56

In [14]:
for pdf in pdfs:
    shutil.copy('../data/PDFs/' + pdf, '../data/mal_pdfs/' + pdf)